<a href="https://colab.research.google.com/github/barikbin/Generative-Adversarial-NN/blob/main/GAN_Initial_Model_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:

import os 
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# Creating variables
s = tf.Variable(2, name='scalar')
m = tf.Variable([[0,1], [2,3]], name='matrix')
W = tf.Variable(tf.zeros([784, 10]), name='big_matix')

# This is an update for the tensorflow version 2
V = tf.Variable(tf.random.truncated_normal([784, 10]), name='normal_matrix')

# We basically assign different types of data here, but newer version of tensoflow has a tottaly diffrenet 
# Syntax structure! 

# s = tf.compat.v1.get_variable('scalar', initializer = tf.constant(2))
# m = tf.Variable('matrix', initializer = tf.constant([[0,1],[2,3]]))
# W = tf.Variable('big_matrix', shape=(784, 10), initializer = tf.zeros_initializer())
# V = tf.Variable('normal_matrix', shape(784, 10), initializer = tf.truncated_normal_initializer())


In [17]:
# So we area certain that global value initializer works!

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(V.eval())


[[ 0.35104892  0.81174916 -0.17623709 ...  0.42579114 -0.29526326
  -0.38179237]
 [ 1.1842904   1.3275487   0.04810633 ...  0.3891088   0.53070134
   0.6285195 ]
 [-1.4645586  -0.43222392 -0.04348407 ...  0.02052263 -0.01252622
   0.11552641]
 ...
 [ 0.17021734  0.35921112 -1.0290936  ...  0.6966726  -0.02934859
  -0.47670984]
 [ 0.7784154   1.6043116   0.24209075 ...  0.13717514  1.0496029
  -1.690364  ]
 [-0.39894167  1.0952942  -0.34575468 ...  0.38497066  0.0392189
  -0.32174274]]


In [19]:

# assigning values to the variables 

W = tf.Variable(10)
W.assign(100)
with tf.Session() as sess:
  sess.run(W.initializer)
  print(sess.run(W))

# We are sure that initializer works in this version of tensorflow!


10


In [20]:
 # Pay close attenstion to the syntax

 W = tf.Variable(10)
 assign_op = W.assign(100)
 with tf.Session() as sess:
    sess.run(assign_op)
    print(W.eval())

# We revoke the value assigned! not the initialized value!    

100


In [23]:

# Create a varibale whose original value is 2
# Well basically we do not have intilizaer in the newer version of tensorflow!
# a = tf.get_variable('scalar', initializer = tf.constant(2))

'''
a_times_two = a.assign( a * 2)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(a_times_two)
  sess.run(a_times_two)
  sess.run(a_times_two)

'''  

'\na_times_two = a.assign( a * 2)\nwith tf.Session() as sess:\n  sess.run(tf.global_variables_initializer())\n  sess.run(a_times_two)\n  sess.run(a_times_two)\n  sess.run(a_times_two)\n\n'

In [28]:

W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10)))
print(sess2.run(W.assign_sub(2)))
print(sess1.run(W.assign_add(100)))
print(sess2.run(W.assign_sub(50)))
sess1.close()
sess2.close()

# Pay attention to the order of calculations! it is not linear!
W = tf.Variable(tf.truncated_normal([700, 10]))

# Need to know what truncated normal is!?
# This is a neural network structure 
U = tf.Variable( W * 2)
print(U)

W = tf.Variable(tf.truncated_normal([700,10]))
U = tf.Variable( W * 2)


20
8
120
-42
<tf.Variable 'Variable_12:0' shape=(700, 10) dtype=float32_ref>


## Application on the Dataset, linear regression example using tf.data

In [31]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import utils

DATA_FILE = './Prediction Dataset USA.txt'

# Here we read the data
# data, n_samples = utils.read_birth_life_data(DATA_FILER)

# Here we create Dataset and iterator
# The methods all come from the util file! that's another file

dataset = tf.data.Dataset.from_tensor_slices((data[:,0], data[:,1]))

iterator = dataset.make_initializable_iterator()
X, Y = iterator.get_next()

# Here we create weight and bias, they are initialized to 0
w = tf.get_variable('weights', initializer = tf.constant(0.0))

b = tf.get_variable('bias', initializer=tf.constant(0.0))

# This is a simple regression and we build the model here!
Y_predicted = X * w + b

# Step 5: use the aquare as the loss function
loss = tf.square(Y - Y_predicted, name='loss')

# Step 6: this is the gradient descent with learning rate of 0.001 to minimize loss
# Pay attenstion to the syntax

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

# Here we initialize the necessary variables, in this case w and b
# Pay attenstion to the time and write syntax

start = time.time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./graphs/linear_reg, sess.graph')

# Here we train the model for 100 epochs
for i in range(100):
    sess.run(iterator.initializer) 
    total_loss = 0
    try:
        while True:  
          # The underscore here is not clear
          _, l = sess.run([optimizer, loss])
          total_loss += l
    except tf.errors.OutOfRangeError:
        pass  

    print('Epoch {0}:{1}' . format(i, total_loss/n_samples)) 

    # We should always close the writer when you are done using it! so
    writer.close()  

# Here we print out the values of w and b
    w_out, b_out = sess.run([w,b])
    print('w:%f, b:%f' %(w_out, b_out))

# we should print the time It took to converge       
print('Took: %f seconds' %(time.time() - start))  

# The results plots
# Pay attention to the syntax of the plot 

plt.plot(data[:,0], data[:,1], 'bo', label='Real data')
plt.plot(data[:,0], data[:,0] * w_out + b_out, 'r', label='Predicted data with squared error')
